In [6]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
import unicodedata
import API_keys

## Scrapping for Biggest European cities

In [7]:

url = ("https://en.wikipedia.org/wiki/List_of_European_cities_by_population_within_city_limits") 
response = requests.get(url)
time.sleep(2)
soup = BeautifulSoup(response.content, "html.parser")
parent = soup.find(class_ = 'wikitable sortable')

children = parent.contents[1]

dictionary = {"City": [],"Country":[],"Population":[]}
for i, child in enumerate(children):
    if((i != 0) & (i%2 == 0)):
        dictionary['City'].append(child.contents[3].get_text("|", strip=True).replace('\n',''))
        dictionary['Population'].append(child.contents[7].get_text("|", strip=True))
        dictionary['Country'].append(child.contents[5].get_text("|", strip=True))
    if(i == 40):
        break

table = pd.DataFrame.from_dict(dictionary)
table.Population = pd.to_numeric(table.Population.str.split('|').str[0].str.replace(',',''))
table.City = table.City.str.split('|').str[0]


## Scraping for country codes

In [8]:
url = ("https://www.iban.com/country-codes") 
response = requests.get(url)
time.sleep(2)
soup = BeautifulSoup(response.content, "html.parser")

In [9]:
def get_country_codes(soup):
    dictionary = {"Country":[],"Country_code":[]}
    table_html = soup.find('table',id="myTable")
    for row in table_html.find_all('tr'):
        for j, column in enumerate(row.find_all('td')):
            if (j==0):
                dictionary['Country'].append(column.get_text())
            if(j==1):
                dictionary['Country_code'].append(column.get_text())
    table_country_codes = pd.DataFrame.from_dict(dictionary)
    return table_country_codes

In [10]:
table_country_codes = get_country_codes(soup)

## Merging cities and country codes

In [11]:
table.Country = table.Country.apply(lambda x:  table_country_codes[table_country_codes.Country.str.contains(x)].Country.values[0])

big_table = table.merge(table_country_codes,how='left')

big_table['CityCountry'] = big_table[['City','Country_code']].apply(lambda x: ', '.join(x), axis = 1)

## Merging with airport codes from airports.csv

In [12]:
airports = pd.read_csv('data/airports.csv')

airports.loc[airports['municipality'] == 'St. Petersburg', 'municipality'] = 'Saint Petersburg'

airports.loc[airports['municipality'] == 'Kiev', 'municipality'] = 'Kyiv'

airports = airports.dropna(subset=['municipality', 'iso_country'])

airports.loc[airports['municipality'].str.contains('Istanbul'), 'municipality'] = 'Istanbul'

list_of_cities = big_table.CityCountry.to_list()

airports_only_needed = airports[['type','name','iso_country','municipality','scheduled_service','gps_code']]

airports_only_needed = (
        airports_only_needed.query("(type == 'medium_airport' | type == 'large_airport') & scheduled_service == 'yes'")
)

big_table['CityCountry'] = big_table[['City','Country_code']].apply(lambda x: ', '.join(x), axis = 1)

airports_only_needed['CityCountry'] = airports_only_needed[['municipality','iso_country']].apply(lambda x: ', '.join(x), axis = 1)

airports_only_needed = airports_only_needed[airports_only_needed.CityCountry.isin(list_of_cities)]

airports_only_needed.type = pd.Categorical(airports_only_needed.type, categories=["large_airport","medium_airport"],ordered=True)

airports_only_needed.sort_values('type', inplace=True)

airports_only_needed = airports_only_needed.drop_duplicates(subset=['CityCountry'])


final_table = big_table.merge(airports_only_needed,how='left').drop(['type','municipality','scheduled_service','iso_country'],axis=1).rename(columns={'name': "Airport_name",'gps_code':"icao"})

In [25]:
def create_mysql_table(big_table):
    schema="gans"
    host="wbs-guns-project.cthpzqamxucy.us-east-2.rds.amazonaws.com"
    user="admin"
    password=API_keys.mysqlPassword
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    big_table.to_sql('cities',con=con,if_exists='append',index=False)

In [26]:
create_mysql_table(final_table)

In [24]:
final_table

,City,Country,Population,Country_code,CityCountry,Airport_name,icao
0,Istanbul,Turkey,15462452,TR,"Istanbul, TR",İstanbul Airport,LTFM
1,Moscow,Russian Federation (the),12195221,RU,"Moscow, RU",Domodedovo International Airport,UUDD
2,London,United Kingdom of Great Britain and Northern I...,9126366,GB,"London, GB",London Luton Airport,EGGW
3,Saint Petersburg,Russian Federation (the),5383890,RU,"Saint Petersburg, RU",Pulkovo Airport,ULLI
4,Berlin,Germany,3748148,DE,"Berlin, DE",Berlin Brandenburg Airport,EDDB
5,Madrid,Spain,3223334,ES,"Madrid, ES",Adolfo Suárez Madrid–Barajas Airport,LEMD
6,Kyiv,Ukraine,2950800,UA,"Kyiv, UA",Boryspil International Airport,UKBB
7,Rome,Italy,2844750,IT,"Rome, IT",Rome–Fiumicino Leonardo da Vinci International...,LIRF
8,Bucharest,Romania,2155240,RO,"Bucharest, RO",Henri Coandă International Airport,LROP
9,Paris,France,2140526,FR,"Paris, FR",Paris-Orly Airport,LFPO


## Creating csv file

In [41]:
# final_table.to_csv('data/cities.csv',index=False)